<a href="https://colab.research.google.com/github/ArosevMaxim/PDA_SOL/blob/main/DFMA1bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 -q install nest_asyncio
!pip3 -q install aiogram

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [14]:
import logging
import asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.filters import CommandStart, Command, StateFilter
from aiogram.fsm.state import State, StatesGroup
from aiogram.enums import ParseMode
from aiogram.types import InlineKeyboardButton, InlineKeyboardMarkup, FSInputFile, KeyboardButton
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
from aiogram.fsm.context import FSMContext
from aiogram import F
import uuid
import sys
import time, json, io

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [21]:
TOKEN = "6704974851:AAF12_-5FaZqc3pEwhKqdi7qXaVUhBF-QJg"
logging.basicConfig(level=logging.INFO, stream=sys.stdout)


dp = Dispatcher()
bot = Bot(TOKEN, parse_mode=ParseMode.HTML)

# Переменные для сообщения "О нас"
photo = FSInputFile('/content/drive/MyDrive/DFMA/DFMA1.jpg') # Фото
caption = "<b>Это демо бот</b>\nЕго цель - презентовать возможности библиотеки aiogram\n\n<i>Сделал Аросев Максим</i>" # Текст с фото, с жирным шрифтом и курсивом

available_genders = ['мужчина', 'женщина']

class Gender(StatesGroup):
    waiting_for_gender = State()

IKB1 = InlineKeyboardBuilder()
Btn1 = types.InlineKeyboardButton(text="О нас", callback_data="btn1")
Btn2 = types.InlineKeyboardButton(text="Ваш пол", callback_data="btn2")
IKB1.add(Btn1, Btn2)


kbm = [[types.KeyboardButton(text="Побриться"),
       types.KeyboardButton(text="Бухнуть")],]
keyboardm = types.ReplyKeyboardMarkup(
        keyboard=kbm,
        resize_keyboard=True,
        input_field_placeholder="Выберите действие")

kbf = [[types.KeyboardButton(text="Купить духи"),
       types.KeyboardButton(text="Сходить на маникюр")],]
keyboardf = types.ReplyKeyboardMarkup(
        keyboard=kbf,
        resize_keyboard=True,
        input_field_placeholder="Выберите действие")


@dp.message(CommandStart())
async def send_welcome(message: types.Message):
    await message.answer(f'Здравствуй {message.from_user.first_name}. \n\nВыбери одну из двух кнопок:', reply_markup=IKB1.as_markup())

@dp.callback_query(F.data.startswith("btn"))
async def process_callback_button(callback_query: types.CallbackQuery, state: FSMContext):
    data = callback_query.data
    if data == 'btn1':
        await bot.send_photo(callback_query.from_user.id, photo, caption = caption, parse_mode=ParseMode.HTML) # Отправляем сообщение с фото, жирным шрифтом
    elif data == 'btn2':
        await bot.send_message(callback_query.from_user.id, 'Введите ваш пол')
        await state.set_state(Gender.waiting_for_gender)

@dp.message(Gender.waiting_for_gender, F.text.in_(available_genders))
async def process_gender(message: types.Message, state: FSMContext):
    await state.update_data(chosen_gender=message.text.lower())
    if message.text == "мужчина":
        await message.answer("Что вы хотете сделать?", reply_markup=keyboardm)
        await state.set_state()
    elif message.text == "женщина":
        await message.answer("Что вы хотите сделать?", reply_markup=keyboardf)
        await state.set_state()
        return


@dp.message(StateFilter("Gender:waiting_for_gender"))
async def gender_incorrectly(message: types.Message):
    await message.answer(text="Пол введен неверно, введите одно из двух: мужчина или женщина")

async def main() -> None:
    # And the run events dispatching
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())